In [21]:
import pandas as pd 
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt 

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from tqdm import tqdm

In [22]:
class Model1 (nn.Module):
    def __init__(self):
        super(Model1,self).__init__()
        self.model = nn.Sequential(
            nn.Linear(5,20),
            nn.LeakyReLU(negative_slope=0.1),
            nn.Linear(20,10),
            nn.LeakyReLU(negative_slope=0.1),
            nn.Linear(10,1),
            nn.Tanh()
        )
    def forward(self,x):
        return self.model(x)

In [23]:
df = pd.read_json("data3/normal_vpm.json",orient="records",lines=True)


In [24]:
list_data = df.to_numpy()
np.random.shuffle(list_data)
test_size = 0.1
test_split = int((1-test_size)*list_data.size)

data_x = torch.Tensor(list_data[:,:-1])
data_Y = torch.Tensor(list_data[:,-1].reshape(-1,1))
data = TensorDataset(data_x,data_Y)
train_data, test_dataset = torch.utils.data.random_split(data,[18000,2348])

train_dataLoader = DataLoader(train_data,batch_size=128,shuffle=True)
test_dataLoader = DataLoader(test_dataset,batch_size=128,shuffle=True)


In [25]:
model1 = Model1()
optimizer = optim.Adam(model1.parameters(),lr=0.01)
criterion = nn.MSELoss()

In [26]:
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")
model1.to(device)
print("Running model on:",device)

Running model on: cpu


In [27]:
epochs = 50

model1.train()
for epoch in tqdm(range(epochs)):
    train_loss = []
    for x, Y in train_dataLoader:
        x = x.to(device)
        Y = Y.to(device)
        optimizer.zero_grad()
        out = model1(x)
        loss = criterion(out, Y)
        loss.backward()
        optimizer.step()
        train_loss.append(loss.item())
    print(f"Loss: {np.average(train_loss):.3f}")
torch.save(model1.state_dict(), "model")

100%|██████████| 50/50 [00:13<00:00,  3.82it/s]Loss: 0.166



In [28]:
correct = 0
errors = []
for x, Y in test_dataLoader:
    model1.eval()
    x = x.to(device)
    Y = Y.cpu()
    out=model1(x)
    out.cpu()
    for i,o in enumerate(out):
        if o * Y[i] > 0.8:
            correct += 1
        else:
            errors.append([x[i],Y[i]])
print(f'Corectly predicted: {correct} out of {len(test_dataset)}\nTest accuracy: {correct/len(test_dataset):.2%}')


Corectly predicted: 1893 out of 2348
Test accuracy: 80.62%


Przykładowa wartość dla której nie ma zakupu, zobaczymy czy dzięki naszemu systemowi byli byśmy wstanie przekonać do zakupu
{"d_time":0.100068132,"n_visits":2,"visits_per_minute":0.0001321401,"max_disc":0,"curr_disc":0,"is_purchased":-1}

In [29]:
model1.cpu()
print(model1(torch.Tensor(np.array([[0.100068132,2,0.0001321401,0,0],[0.100068132,2,0.0001321401,0,5],[0.100068132,2,0.0001321401,0,10],[0.100068132,2,0.0001321401,0,15],[0.100068132,2,0.0001321401,0,20]]))))
#print(model1(torch.Tensor(np.array([60000000,2,0,0]))))

tensor([[-0.9910],
        [-0.9754],
        [-0.9270],
        [-0.4369],
        [ 0.6954]], grad_fn=<TanhBackward>)


In [30]:
errors

00, 0.0000e+00]),
  tensor([1.])],
 [tensor([2.2988e-01, 4.0000e+00, 1.1504e-04, 5.0000e+00, 0.0000e+00]),
  tensor([-1.])],
 [tensor([3.2709e-01, 5.0000e+00, 1.0107e-04, 1.0000e+01, 1.0000e+01]),
  tensor([-1.])],
 [tensor([3.6104e-01, 4.0000e+00, 7.3250e-05, 1.0000e+01, 1.0000e+01]),
  tensor([-1.])],
 [tensor([5.6930e-01, 3.0000e+00, 3.4840e-05, 0.0000e+00, 0.0000e+00]),
  tensor([-1.])],
 [tensor([4.0150e-01, 6.0000e+00, 9.8802e-05, 1.0000e+01, 1.0000e+01]),
  tensor([1.])],
 [tensor([6.0106e-01, 5.0000e+00, 5.4999e-05, 2.0000e+01, 2.0000e+01]),
  tensor([-1.])],
 [tensor([5.6225e-01, 4.0000e+00, 4.7036e-05, 5.0000e+00, 5.0000e+00]),
  tensor([-1.])],
 [tensor([4.3756e-01, 7.0000e+00, 1.0577e-04, 1.0000e+01, 0.0000e+00]),
  tensor([-1.])],
 [tensor([2.8848e-01, 4.0000e+00, 9.1674e-05, 1.5000e+01, 1.5000e+01]),
  tensor([1.])],
 [tensor([3.7281e-01, 6.0000e+00, 1.0641e-04, 1.0000e+01, 0.0000e+00]),
  tensor([-1.])],
 [tensor([7.1493e-02, 3.0000e+00, 2.7743e-04, 1.0000e+01, 0.0000e+0